# Interfaz de Base de Datos para una cadena de Supermercados

Se arma una interfaz de sirve para controlar el stock que se tiene de cada producto en las distintas sucursales que tiene esta cadena de supermercado. Ademas se crean las bases de datos necesarias para trabar con esta nueva interfaz.

In [ ]:
#Librerias a utilizar en el proyecto
import pandas as pd
import sqlite3
from ttkthemes import ThemedTk
from tkinter import ttk, messagebox
import tkinter as tk


Creo y Armo la base de datos
La misma se va a crear en donde este alojado el script 
Esta base se nutre de informacion de un excel que posee registros necesarios para la correspondiente practica

In [ ]:
# Crea & conecta a la base de datos
conn = sqlite3.connect('supermercado.db')
cursor = conn.cursor()

# Crea tabla Sucursal
cursor.execute('''
CREATE TABLE IF NOT EXISTS Sucursal (
    ID INTEGER PRIMARY KEY AUTOINCREMENT,
    Nombre TEXT NOT NULL,
    Direccion TEXT NOT NULL,
    Altura INTEGER,
    Distrito TEXT
)
''')

# Crea tabla Rubro
cursor.execute('''
CREATE TABLE IF NOT EXISTS Rubro (
    ID INTEGER PRIMARY KEY AUTOINCREMENT,
    Nombre TEXT NOT NULL,
    Descripcion TEXT
)
''')

# Crea tabla Productos
cursor.execute('''
CREATE TABLE IF NOT EXISTS Productos (
    ID INTEGER PRIMARY KEY AUTOINCREMENT,
    Nombre TEXT NOT NULL,
    Cod_Rubro INTEGER NOT NULL,
    Marca TEXT,
    Precio REAL NOT NULL,
    Cantidad INTEGER NOT NULL,
    Cod_Sucursal INTEGER NOT NULL,
    FOREIGN KEY (Cod_Rubro) REFERENCES Rubro(ID),
    FOREIGN KEY (Cod_Sucursal) REFERENCES Sucursal(ID)
)
''')

# Lee archivo Excel
archivo = "C:\\Users\\Cristian\\Desktop\\Tablas_CRUD.xlsx" #Colocar la direccion de donde este alojado el excel Base

sucursales_df = pd.read_excel(archivo, sheet_name='Sucursal')
rubros_df = pd.read_excel(archivo, sheet_name='Rubro')
productos_df = pd.read_excel(archivo, sheet_name='Productos')

#--Pequeña  Transformacion de Datos---
sucursales_df.columns = sucursales_df.columns.str.strip()
rubros_df.columns = rubros_df.columns.str.strip()
productos_df.columns = productos_df.columns.str.strip()

# Conecto a la base de datos
conn = sqlite3.connect('supermercado.db')
cursor = conn.cursor()

# Limpio tablas antes de cargar
cursor.execute("DELETE FROM Productos")
cursor.execute("DELETE FROM Sucursal")
cursor.execute("DELETE FROM Rubro")
conn.commit()

# Resetea autoincrementos
cursor.execute("DELETE FROM sqlite_sequence WHERE name='Sucursal'")
cursor.execute("DELETE FROM sqlite_sequence WHERE name='Rubro'")
cursor.execute("DELETE FROM sqlite_sequence WHERE name='Productos'")
conn.commit()

# Inserta Sucursales
for _, row in sucursales_df.iterrows():
    cursor.execute("""
        INSERT INTO Sucursal (ID, Nombre, Direccion, Altura, Distrito)
        VALUES (?, ?, ?, ?, ?)
    """, (row['ID'], row['Nombre'], row['Direccion'], row['Altura'], row['Distrito']))

# Inserta Rubros
for _, row in rubros_df.iterrows():
    cursor.execute("""
        INSERT INTO Rubro (ID, Nombre, Descripcion)
        VALUES (?, ?, ?)
    """, (row['ID'], row['Nombre'], row['Descripcion']))

# Inserta Productos
for _, row in productos_df.iterrows():
    cursor.execute("""
        INSERT INTO Productos (ID, Nombre, Cod_Rubro, Marca, Precio, Cantidad, Cod_Sucursal)
        VALUES (?, ?, ?, ?, ?, ?, ?)
    """, (
        row['ID'],
        row['Nombre'],
        row['ID_Rubro'],
        row['Marca'],
        row['Precio'],
        row['Cantidad'],
        row['ID_Sucursal']
    ))
# Confirmar cambios y cerrar conexión
conn.commit()
conn.close()

Consulto a la base para ver si los datos fueron cargados correctamente

In [5]:
conn = sqlite3.connect('supermercado.db')
productos = pd.read_sql_query("SELECT * FROM Productos", conn)
print(productos)
sucursales = pd.read_sql_query("SELECT * FROM Sucursal", conn)
print(sucursales)
rubros = pd.read_sql_query("SELECT * FROM Rubro", conn)
print(rubros)
conn.close()

    ID                             Nombre  Cod_Rubro       Marca    Precio  \
0    1                    Veranda  x250gr          1  Starbucks   14946.40   
1    2                    Veranda  x250gr          1  Starbucks   14946.40   
2    3        Nescafe Dolca Latte x 125gr          1      Nestle   3809.00   
3    4        Nescafe Dolca Latte x 125gr          1      Nestle   3809.00   
4    5           Café Torrado Molido 500g          1    Bonafide   8649.75   
5    6           Café Torrado Molido 500g          1    Bonafide   8649.75   
6    7              cafe tostado en grano          1    Cabrales  21226.00   
7    8              cafe tostado en grano          1    Cabrales  21226.00   
8    9       yogur con granola yogurisimo          2      Danone   3650.00   
9   10                manteca coto 200 gr          2        Coto   2900.00   
10  11                manteca coto 200 gr          2        Coto   2900.00   
11  12  leche fermentada frutilla actimel          2  Serenisima

Ya con los datos cargados correctamente paso a crear la interfaz para el CRUD

In [ ]:
# ------------------- VENTANA PRINCIPAL Y FONDO --------------------
root = ThemedTk(theme="xp") #Se provo con varios temas pero ninguno lograba funcionar correctamente
root.title("Gestión de Supermercado")
root.geometry("800x600")
root.iconbitmap(r"C:\Users\Cristian\Desktop\Clase de Ingles\icono.ico") #Colocar la direccion donde este alojado el icono

# Estilos visuales
style = ttk.Style()
style.configure('TButton', padding=5, relief="flat")
style.configure('TLabel', padding=3)
style.configure('Treeview', rowheight=25)

notebook = ttk.Notebook(root)
notebook.pack(fill='both', expand=True)

# --------------- FUNCIONES BASE DE DATOS (solo de Productos) ---------------

def get_rubros():
    conn = sqlite3.connect('supermercado.db')
    cur = conn.cursor()
    cur.execute("SELECT ID, Nombre FROM Rubro")
    rubros = cur.fetchall()
    conn.close()
    return rubros

def get_rubros_full():
    conn = sqlite3.connect('supermercado.db')
    cur = conn.cursor()
    cur.execute("SELECT ID, Nombre, Descripcion FROM Rubro")
    rubros = cur.fetchall()
    conn.close()
    return rubros

def add_rubro(nombre, descripcion):
    conn = sqlite3.connect('supermercado.db')
    cur = conn.cursor()
    cur.execute("INSERT INTO Rubro (Nombre, Descripcion) VALUES (?, ?)", (nombre, descripcion))
    conn.commit()
    conn.close()

def update_rubro(id_rubro, nombre, descripcion):
    conn = sqlite3.connect('supermercado.db')
    cur = conn.cursor()
    cur.execute("UPDATE Rubro SET Nombre=?, Descripcion=? WHERE ID=?", (nombre, descripcion, id_rubro))
    conn.commit()
    conn.close()

def delete_rubro(id_rubro):
    conn = sqlite3.connect('supermercado.db')
    cur = conn.cursor()
    cur.execute("DELETE FROM Rubro WHERE ID=?", (id_rubro,))
    conn.commit()
    conn.close()

def get_sucursales():
    conn = sqlite3.connect('supermercado.db')
    cur = conn.cursor()
    cur.execute("SELECT ID, Nombre FROM Sucursal")
    sucursales = cur.fetchall()
    conn.close()
    return sucursales

def get_sucursales_full():
    conn = sqlite3.connect('supermercado.db')
    cur = conn.cursor()
    cur.execute("SELECT ID, Nombre, Direccion, Altura, Distrito FROM Sucursal")
    sucursales = cur.fetchall()
    conn.close()
    return sucursales

def add_sucursal(nombre, direccion, altura, distrito):
    conn = sqlite3.connect('supermercado.db')
    cur = conn.cursor()
    cur.execute("INSERT INTO Sucursal (Nombre, Direccion, Altura, Distrito) VALUES (?, ?, ?, ?)", (nombre, direccion, altura, distrito))
    conn.commit()
    conn.close()

def update_sucursal(id_suc, nombre, direccion, altura, distrito):
    conn = sqlite3.connect('supermercado.db')
    cur = conn.cursor()
    cur.execute("UPDATE Sucursal SET Nombre=?, Direccion=?, Altura=?, Distrito=? WHERE ID=?", (nombre, direccion, altura, distrito, id_suc))
    conn.commit()
    conn.close()

def delete_sucursal(id_suc):
    conn = sqlite3.connect('supermercado.db')
    cur = conn.cursor()
    cur.execute("DELETE FROM Sucursal WHERE ID=?", (id_suc,))
    conn.commit()
    conn.close()

def get_productos():
    conn = sqlite3.connect('supermercado.db')
    cur = conn.cursor()
    cur.execute("""
        SELECT Productos.ID, Productos.Nombre, Rubro.Nombre, Productos.Marca, Productos.Precio, Productos.Cantidad, Sucursal.Nombre
        FROM Productos JOIN Rubro ON Productos.Cod_Rubro = Rubro.ID JOIN Sucursal ON Productos.Cod_Sucursal = Sucursal.ID""")
    productos = cur.fetchall()
    conn.close()
    return productos

def add_producto(nombre, id_rubro, marca, precio, cantidad, id_sucursal):
    conn = sqlite3.connect('supermercado.db')
    cur = conn.cursor()
    cur.execute("INSERT INTO Productos (Nombre, Cod_Rubro, Marca, Precio, Cantidad, Cod_Sucursal) VALUES (?, ?, ?, ?, ?, ?)",
                (nombre, id_rubro, marca, precio, cantidad, id_sucursal))
    conn.commit()
    conn.close()

def update_producto(id_prod, nombre, id_rubro, marca, precio, cantidad, id_sucursal):
    conn = sqlite3.connect('supermercado.db')
    cur = conn.cursor()
    cur.execute("""UPDATE Productos SET Nombre=?, Cod_Rubro=?, Marca=?, Precio=?, Cantidad=?, Cod_Sucursal=? WHERE ID=?""",
                (nombre, id_rubro, marca, precio, cantidad, id_sucursal, id_prod))
    conn.commit()
    conn.close()

def delete_producto(id_prod):
    conn = sqlite3.connect('supermercado.db')
    cur = conn.cursor()
    cur.execute("DELETE FROM Productos WHERE ID=?", (id_prod,))
    conn.commit()
    conn.close()

# ----------------------- PESTAÑA PRODUCTOS ----------------------

frame_productos = ttk.Frame(notebook)
notebook.add(frame_productos, text='Productos')

# --- Botonera y búsqueda en la misma fila ---
frame_top_prod = tk.Frame(frame_productos)
frame_top_prod.pack(fill=tk.X, pady=5)

entry_buscar_prod = tk.Entry(frame_top_prod, width=30)
entry_buscar_prod.pack(side=tk.LEFT, padx=5)
btn_buscar_prod = tk.Button(frame_top_prod, text="Buscar")
btn_buscar_prod.pack(side=tk.LEFT, padx=5)

btn_guardar = tk.Button(frame_top_prod, text="Agregar producto")
btn_guardar.pack(side=tk.LEFT, padx=5)
btn_nuevo = tk.Button(frame_top_prod, text="Nuevo producto")
btn_nuevo.pack(side=tk.LEFT, padx=5)
btn_eliminar = tk.Button(frame_top_prod, text="Eliminar producto")
btn_eliminar.pack(side=tk.LEFT, padx=5)
btn_limpiar = tk.Button(frame_top_prod, text="Limpiar formulario")
btn_limpiar.pack(side=tk.LEFT, padx=5)

# Filtros
frame_filtros = tk.Frame(frame_productos)
frame_filtros.pack(fill=tk.X, pady=5)

filtro_rubro = ttk.Combobox(frame_filtros, state="readonly", width=15)
filtro_rubro['values'] = ["Todos"] + [r[1] for r in get_rubros()]
filtro_rubro.set("Todos")
filtro_rubro.pack(side=tk.LEFT, padx=5)

filtro_sucursal = ttk.Combobox(frame_filtros, state="readonly", width=15)
filtro_sucursal['values'] = ["Todas"] + [s[1] for s in get_sucursales()]
filtro_sucursal.set("Todas")
filtro_sucursal.pack(side=tk.LEFT, padx=5)

def limpiar_filtros():
    filtro_rubro.set("Todos")
    filtro_sucursal.set("Todas")
    actualizar_tabla()

tk.Button(frame_filtros, text="Limpiar filtros", command=limpiar_filtros).pack(side=tk.LEFT, padx=5)

# Formulario productos
form_p = tk.Frame(frame_productos, padx=10, pady=10)
form_p.pack(side=tk.TOP, fill=tk.X)

labels = ["Nombre:", "Rubro:", "Marca:", "Precio:", "Cantidad:", "Sucursal:"]
entries = [tk.Entry(form_p) if i!=1 and i!=5 else ttk.Combobox(form_p, state="readonly") for i in range(6)]

for i, label in enumerate(labels):
    tk.Label(form_p, text=label).grid(row=i//2, column=(i%2)*2)
    entries[i].grid(row=i//2, column=(i%2)*2+1)

entry_nombre, combobox_rubro, entry_marca, entry_precio, entry_cantidad, combobox_sucursal = entries

tree = ttk.Treeview(frame_productos, columns=("ID", "Nombre", "Rubro", "Marca", "Precio", "Cantidad", "Sucursal"), show='headings')
for col in tree["columns"]:
    tree.heading(col, text=col)
tree.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
tree.tag_configure('stock_bajo', background='#ffcccc')

# --- Funciones CRUD de productos
def limpiar_formulario():
    entry_nombre.delete(0, tk.END)
    combobox_rubro.set('')
    entry_marca.delete(0, tk.END)
    entry_precio.delete(0, tk.END)
    entry_cantidad.delete(0, tk.END)
    combobox_sucursal.set('')
    btn_guardar.config(text="Agregar producto")
    tree.selection_remove(tree.selection())
    entry_buscar_prod.delete(0, tk.END)  # <--- Limpia el campo de búsqueda
    actualizar_tabla()  # <--- Restaura la tabla completa


def nuevo_producto():
    tree.selection_remove(tree.selection())
    limpiar_formulario()

def guardar_producto():
    nombre = entry_nombre.get().strip()
    marca = entry_marca.get().strip()
    precio = entry_precio.get().strip()
    cantidad = entry_cantidad.get().strip()
    rubro_idx = combobox_rubro.current()
    sucursal_idx = combobox_sucursal.current()
    if not nombre or rubro_idx==-1 or not precio or not cantidad or sucursal_idx==-1:
        messagebox.showwarning("Campos incompletos", "Completa todos los campos requeridos.")
        return
    try:
        precio = float(precio)
        cantidad = int(cantidad)
    except ValueError:
        messagebox.showwarning("Valores inválidos", "Precio y cantidad deben ser numéricos.")
        return
    id_rubro = get_rubros()[rubro_idx][0]
    id_sucursal = get_sucursales()[sucursal_idx][0]
    if btn_guardar['text'] == "Actualizar producto":
        id_prod = tree.item(tree.selection())['values'][0]
        update_producto(id_prod, nombre, id_rubro, marca, precio, cantidad, id_sucursal)
        messagebox.showinfo("Actualizado", "Producto actualizado correctamente.")
    else:
        add_producto(nombre, id_rubro, marca, precio, cantidad, id_sucursal)
        messagebox.showinfo("Agregado", "Producto agregado correctamente.")
    limpiar_formulario()
    actualizar_tabla()

btn_guardar.config(command=guardar_producto)
btn_nuevo.config(command=nuevo_producto)
btn_eliminar.config(command=lambda: eliminar_producto())
btn_limpiar.config(command=limpiar_formulario)

def seleccionar_producto(event):
    selected = tree.selection()
    if selected:
        values = tree.item(selected)['values']
        entry_nombre.delete(0, tk.END)
        entry_nombre.insert(0, values[1])
        combobox_rubro.set(values[2])
        entry_marca.delete(0, tk.END)
        entry_marca.insert(0, values[3])
        entry_precio.delete(0, tk.END)
        entry_precio.insert(0, str(values[4]))
        entry_cantidad.delete(0, tk.END)
        entry_cantidad.insert(0, str(values[5]))
        combobox_sucursal.set(values[6])
        btn_guardar.config(text="Actualizar producto")
    else:
        limpiar_formulario()
tree.bind("<<TreeviewSelect>>", seleccionar_producto)

def eliminar_producto():
    selected = tree.selection()
    if not selected:
        messagebox.showwarning("Selecciona un producto", "Primero selecciona un producto para eliminar.")
        return
    id_prod = tree.item(selected)['values'][0]
    if messagebox.askyesno("Confirmar", "¿Seguro que deseas eliminar este producto?"):
        delete_producto(id_prod)
        limpiar_formulario()
        actualizar_tabla()

# ----------- Búsqueda de productos por nombre o marca ----------
def buscar_producto():
    texto = entry_buscar_prod.get().strip().lower()
    for row in tree.get_children():
        tree.delete(row)

    conn = sqlite3.connect('supermercado.db')
    sql = """
        SELECT Productos.ID, Productos.Nombre, Rubro.Nombre, Productos.Marca, Productos.Precio, Productos.Cantidad, Sucursal.Nombre
        FROM Productos 
        JOIN Rubro ON Productos.Cod_Rubro = Rubro.ID 
        JOIN Sucursal ON Productos.Cod_Sucursal = Sucursal.ID
        WHERE LOWER(Productos.Nombre) LIKE ? OR LOWER(Productos.Marca) LIKE ?
    """
    productos = conn.execute(sql, (f'%{texto}%', f'%{texto}%')).fetchall()
    conn.close()

    for row in productos:
        tag = 'stock_bajo' if row[5]<10 else ''
        tree.insert('', tk.END, values=row, tags=(tag,))

btn_buscar_prod.config(command=buscar_producto)

def actualizar_tabla():
    combobox_rubro['values'] = [r[1] for r in get_rubros()]
    combobox_sucursal['values'] = [s[1] for s in get_sucursales()]

    sql = """
        SELECT Productos.ID, Productos.Nombre, Rubro.Nombre, Productos.Marca, Productos.Precio, Productos.Cantidad, Sucursal.Nombre
        FROM Productos JOIN Rubro ON Productos.Cod_Rubro = Rubro.ID JOIN Sucursal ON Productos.Cod_Sucursal = Sucursal.ID"""
    condiciones, params = [], []
    if filtro_rubro.get() != "Todos":
        condiciones.append("Rubro.Nombre=?")
        params.append(filtro_rubro.get())
    if filtro_sucursal.get() != "Todas":
        condiciones.append("Sucursal.Nombre=?")
        params.append(filtro_sucursal.get())
    if condiciones:
        sql += " WHERE " + " AND ".join(condiciones)

    conn = sqlite3.connect('supermercado.db')
    productos = conn.execute(sql, params).fetchall()
    conn.close()

    for row in tree.get_children():
        tree.delete(row)
    for row in productos:
        tag = 'stock_bajo' if row[5]<10 else ''
        tree.insert('', tk.END, values=row, tags=(tag,))

filtro_rubro.bind("<<ComboboxSelected>>", lambda e: actualizar_tabla())
filtro_sucursal.bind("<<ComboboxSelected>>", lambda e: actualizar_tabla())

actualizar_tabla()

# ---------------------- PESTAÑA RUBROS ----------------------

frame_rubros = ttk.Frame(notebook)
notebook.add(frame_rubros, text='Rubros')

form_rubros = tk.Frame(frame_rubros, padx=10, pady=10)
form_rubros.pack(side=tk.TOP, fill=tk.X)

tk.Label(form_rubros, text="Nombre:").grid(row=0, column=0)
entry_nombre_rubro = tk.Entry(form_rubros)
entry_nombre_rubro.grid(row=0, column=1)

tk.Label(form_rubros, text="Descripción:").grid(row=1, column=0)
entry_desc_rubro = tk.Entry(form_rubros)
entry_desc_rubro.grid(row=1, column=1)

frame_botones_rubros = tk.Frame(frame_rubros)
frame_botones_rubros.pack(fill=tk.X, pady=5)

btn_guardar_rubro = tk.Button(frame_botones_rubros, text="Agregar rubro")
btn_guardar_rubro.pack(side=tk.LEFT, padx=5)

btn_nuevo_rubro = tk.Button(frame_botones_rubros, text="Nuevo rubro")
btn_nuevo_rubro.pack(side=tk.LEFT, padx=5)

btn_eliminar_rubro = tk.Button(frame_botones_rubros, text="Eliminar rubro")
btn_eliminar_rubro.pack(side=tk.LEFT, padx=5)

btn_limpiar_rubro = tk.Button(frame_botones_rubros, text="Limpiar formulario")
btn_limpiar_rubro.pack(side=tk.LEFT, padx=5)

tree_rubros = ttk.Treeview(frame_rubros, columns=("ID", "Nombre", "Descripcion"), show='headings')
for col in ("ID", "Nombre", "Descripcion"):
    tree_rubros.heading(col, text=col)
tree_rubros.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)

def limpiar_rubro():
    entry_nombre_rubro.delete(0, tk.END)
    entry_desc_rubro.delete(0, tk.END)
    btn_guardar_rubro.config(text="Agregar rubro")
    tree_rubros.selection_remove(tree_rubros.selection())

def nuevo_rubro():
    tree_rubros.selection_remove(tree_rubros.selection())
    limpiar_rubro()

def guardar_rubro():
    nombre = entry_nombre_rubro.get().strip()
    desc = entry_desc_rubro.get().strip()
    if not nombre:
        messagebox.showwarning("Campos incompletos", "Debes completar al menos el nombre.")
        return
    if btn_guardar_rubro['text'] == "Actualizar rubro":
        id_r = tree_rubros.item(tree_rubros.selection())['values'][0]
        update_rubro(id_r, nombre, desc)
        messagebox.showinfo("Actualizado", "Rubro actualizado.")
    else:
        add_rubro(nombre, desc)
        messagebox.showinfo("Agregado", "Rubro agregado.")
    limpiar_rubro()
    actualizar_tabla_rubro()
    actualizar_tabla()

def seleccionar_rubro(event):
    selected = tree_rubros.selection()
    if selected:
        values = tree_rubros.item(selected)['values']
        entry_nombre_rubro.delete(0, tk.END)
        entry_nombre_rubro.insert(0, values[1])
        entry_desc_rubro.delete(0, tk.END)
        entry_desc_rubro.insert(0, values[2])
        btn_guardar_rubro.config(text="Actualizar rubro")
    else:
        limpiar_rubro()
tree_rubros.bind("<<TreeviewSelect>>", seleccionar_rubro)

def eliminar_rubro():
    selected = tree_rubros.selection()
    if not selected:
        messagebox.showwarning("Selecciona un rubro", "Selecciona un rubro para eliminar.")
        return
    id_r = tree_rubros.item(selected)['values'][0]
    if messagebox.askyesno("Confirmar", "¿Deseas eliminar este rubro?"):
        delete_rubro(id_r)
        limpiar_rubro()
        actualizar_tabla_rubro()
        actualizar_tabla()

btn_guardar_rubro.config(command=guardar_rubro)
btn_nuevo_rubro.config(command=nuevo_rubro)
btn_eliminar_rubro.config(command=eliminar_rubro)
btn_limpiar_rubro.config(command=limpiar_rubro)

def actualizar_tabla_rubro():
    for row in tree_rubros.get_children():
        tree_rubros.delete(row)
    for row in get_rubros_full():
        tree_rubros.insert('', tk.END, values=row)
actualizar_tabla_rubro()

# ---------------------- PESTAÑA SUCURSALES ----------------------

frame_sucursales = ttk.Frame(notebook)
notebook.add(frame_sucursales, text='Sucursales')

form_suc = tk.Frame(frame_sucursales, padx=10, pady=10)
form_suc.pack(side=tk.TOP, fill=tk.X)

labels_suc = ["Nombre:", "Dirección:", "Altura:", "Distrito:"]
entries_suc = [tk.Entry(form_suc) for _ in range(4)]
for i, label in enumerate(labels_suc):
    tk.Label(form_suc, text=label).grid(row=i, column=0)
    entries_suc[i].grid(row=i, column=1)

entry_nombre_suc, entry_dir_suc, entry_altura_suc, entry_dist_suc = entries_suc

# --- Filtro por distrito ---
frame_filtro_distrito = tk.Frame(frame_sucursales)
frame_filtro_distrito.pack(fill=tk.X, pady=5)

def get_distritos():
    conn = sqlite3.connect('supermercado.db')
    cur = conn.cursor()
    cur.execute("SELECT DISTINCT Distrito FROM Sucursal")
    distritos = [row[0] for row in cur.fetchall() if row[0] is not None]
    conn.close()
    return distritos

filtro_distrito = ttk.Combobox(frame_filtro_distrito, state="readonly", width=20)
filtro_distrito.pack(side=tk.LEFT, padx=5)

def actualizar_filtro_distrito():
    distritos = get_distritos()
    filtro_distrito['values'] = ["Todos"] + distritos
    # Si el distrito actual no está en la lista, resetea
    if filtro_distrito.get() not in filtro_distrito['values']:
        filtro_distrito.set("Todos")
actualizar_filtro_distrito()
filtro_distrito.set("Todos")

def limpiar_filtro_distrito():
    filtro_distrito.set("Todos")
    actualizar_tabla_suc()
tk.Button(frame_filtro_distrito, text="Limpiar filtro", command=limpiar_filtro_distrito).pack(side=tk.LEFT, padx=5)

# Evento que actualiza la tabla cuando cambia el distrito
filtro_distrito.bind("<<ComboboxSelected>>", lambda e: actualizar_tabla_suc())

# --- Botones de CRUD ---
frame_botones_suc = tk.Frame(frame_sucursales)
frame_botones_suc.pack(fill=tk.X, pady=5)

btn_guardar_suc = tk.Button(frame_botones_suc, text="Agregar sucursal")
btn_guardar_suc.pack(side=tk.LEFT, padx=5)
btn_nuevo_suc = tk.Button(frame_botones_suc, text="Nueva sucursal")
btn_nuevo_suc.pack(side=tk.LEFT, padx=5)
btn_eliminar_suc = tk.Button(frame_botones_suc, text="Eliminar sucursal")
btn_eliminar_suc.pack(side=tk.LEFT, padx=5)
btn_limpiar_suc = tk.Button(frame_botones_suc, text="Limpiar formulario")
btn_limpiar_suc.pack(side=tk.LEFT, padx=5)

tree_suc = ttk.Treeview(frame_sucursales, columns=("ID", "Nombre", "Direccion", "Altura", "Distrito"), show='headings')
for col in ("ID", "Nombre", "Direccion", "Altura", "Distrito"):
    tree_suc.heading(col, text=col)
tree_suc.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)

def limpiar_suc():
    for e in entries_suc:
        e.delete(0, tk.END)
    btn_guardar_suc.config(text="Agregar sucursal")
    tree_suc.selection_remove(tree_suc.selection())

def nuevo_suc():
    tree_suc.selection_remove(tree_suc.selection())
    limpiar_suc()

def guardar_suc():
    nombre, dir, altura, distrito = [e.get().strip() for e in entries_suc]
    if not nombre or not dir or not altura or not distrito:
        messagebox.showwarning("Campos incompletos", "Completa todos los campos.")
        return
    try:
        altura = int(altura)
    except:
        messagebox.showwarning("Error", "Altura debe ser numérica.")
        return
    if btn_guardar_suc['text']=="Actualizar sucursal":
        id_s = tree_suc.item(tree_suc.selection())['values'][0]
        update_sucursal(id_s, nombre, dir, altura, distrito)
        messagebox.showinfo("Actualizado", "Sucursal actualizada.")
    else:
        add_sucursal(nombre, dir, altura, distrito)
        messagebox.showinfo("Agregado", "Sucursal agregada.")
    limpiar_suc()
    actualizar_tabla_suc()
    actualizar_tabla()
    actualizar_filtro_distrito()  # <-- Esto refresca el filtro

def seleccionar_suc(event):
    selected = tree_suc.selection()
    if selected:
        values = tree_suc.item(selected)['values']
        for i, val in enumerate(values[1:]):
            entries_suc[i].delete(0, tk.END)
            entries_suc[i].insert(0, val)
        btn_guardar_suc.config(text="Actualizar sucursal")
    else:
        limpiar_suc()
tree_suc.bind("<<TreeviewSelect>>", seleccionar_suc)

def eliminar_suc():
    selected = tree_suc.selection()
    if not selected:
        messagebox.showwarning("Selecciona una sucursal", "Primero selecciona una sucursal.")
        return
    id_s = tree_suc.item(selected)['values'][0]
    if messagebox.askyesno("Confirmar", "¿Eliminar sucursal?"):
        delete_sucursal(id_s)
        limpiar_suc()
        actualizar_tabla_suc()
        actualizar_tabla()
        actualizar_filtro_distrito()  # <-- Esto refresca el filtro

btn_guardar_suc.config(command=guardar_suc)
btn_nuevo_suc.config(command=nuevo_suc)
btn_eliminar_suc.config(command=eliminar_suc)
btn_limpiar_suc.config(command=limpiar_suc)

def actualizar_tabla_suc():
    for row in tree_suc.get_children():
        tree_suc.delete(row)
    sql = "SELECT ID, Nombre, Direccion, Altura, Distrito FROM Sucursal"
    params = []
    if filtro_distrito.get() != "Todos":
        sql += " WHERE Distrito=?"
        params.append(filtro_distrito.get())
    conn = sqlite3.connect('supermercado.db')
    sucursales = conn.execute(sql, params).fetchall()
    conn.close()
    for row in sucursales:
        tree_suc.insert('', tk.END, values=row)

actualizar_tabla_suc()

root.mainloop()